In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import datetime as dt
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import os
import re
import requests
import folium

In [2]:
df = pd.read_pickle(r'C:/Users/34670/Desktop/python/Hack a boss/proyecto_3/streamlit/proyecto_aviones/data/pickle/vuelos_limpio.pkl')

In [3]:
# Extraemos las columnas que nos interesan, en este caso, solamente 'aerolinea'
df_aerolineas = df[['aerolinea']].copy()

# Eliminamos los duplicados para asegurar la unicidad de las aerolíneas
df_aerolineas_unicas = df_aerolineas.drop_duplicates()

# Restablecemos el índice
df_aerolineas_unicas.reset_index(drop=True, inplace=True)

# Mostramos el DataFrame resultante
print(df_aerolineas_unicas)


             aerolinea
0            Envoy Air
1     SkyWest Airlines
2         PSA Airlines
3         Endeavor Air
4        Mesa Airlines
5   Southwest Airlines
6        Allegiant Air
7    American Airlines
8      Delta Air Lines
9    Frontier Airlines
10     JetBlue Airways
11    Republic Airways
12     United Airlines
13     Alaska Airlines
14   Hawaiian Airlines
15         Horizon Air
16     Spirit Airlines


In [6]:
import requests
import pandas as pd

def obtener_info_wikipedia(titulo):
    S = requests.Session()
    URL = "https://es.wikipedia.org/w/api.php"

    # Obtener el resumen de la página
    RESUMEN_PARAMS = {
        "action": "query",
        "format": "json",
        "titles": titulo,
        "prop": "extracts",
        "exintro": True,
        "explaintext": True,
    }

    resumen_response = S.get(url=URL, params=RESUMEN_PARAMS)
    resumen_data = resumen_response.json()

    # Obtener la URL de la imagen principal de la página
    IMAGEN_PARAMS = {
        "action": "query",
        "format": "json",
        "titles": titulo,
        "prop": "pageimages",
        "pithumbsize": 500  # Tamaño de la miniatura de la imagen
    }

    imagen_response = S.get(url=URL, params=IMAGEN_PARAMS)
    imagen_data = imagen_response.json()

    resumen = list(resumen_data['query']['pages'].values())[0].get('extract', '')
    imagen_url = list(imagen_data['query']['pages'].values())[0].get('thumbnail', {}).get('source', '')

    return resumen, imagen_url

# Ejemplo de cómo aplicar la función a tu DataFrame df_aerolineas_unicas
for index, row in df_aerolineas_unicas.iterrows():
    resumen, imagen_url = obtener_info_wikipedia(row['aerolinea'])
    df_aerolineas_unicas.at[index, 'Resumen'] = resumen
    df_aerolineas_unicas.at[index, 'Imagen_URL'] = imagen_url

# Verificar el DataFrame actualizado
print(df_aerolineas_unicas.head())


C:\Users\34670\AppData\Local\Temp\ipykernel_14420\4216742747.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aerolineas_unicas.at[index, 'Resumen'] = resumen
C:\Users\34670\AppData\Local\Temp\ipykernel_14420\4216742747.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aerolineas_unicas.at[index, 'Imagen_URL'] = imagen_url


          aerolinea                                            Resumen  \
0         Envoy Air  Envoy Air[2]​ (anteriormente American Eagle Ai...   
1  SkyWest Airlines  SkyWest Airlines, Inc. es una de las dos aerol...   
2      PSA Airlines                                                      
3      Endeavor Air                                                      
4     Mesa Airlines  Mesa Airlines, Inc. (NASDAQ: MESA) es una aero...   

                                          Imagen_URL  
0  https://upload.wikimedia.org/wikipedia/commons...  
1  https://upload.wikimedia.org/wikipedia/commons...  
2                                                     
3                                                     
4  https://upload.wikimedia.org/wikipedia/commons...  


In [4]:
df_aerolineas_unicas.to_pickle('C:/Users/34670/Desktop/python/Hack a boss/proyecto_3/streamlit/proyecto_aviones/data/aerolineas_unicos.pkl')

In [7]:
def obtener_info_wikipedia(titulo, idioma='es'):
    S = requests.Session()
    URL = f"https://{idioma}.wikipedia.org/w/api.php"

    # Obtener el resumen de la página
    RESUMEN_PARAMS = {
        "action": "query",
        "format": "json",
        "titles": titulo,
        "prop": "extracts",
        "exintro": True,
        "explaintext": True,
    }

    resumen_response = S.get(url=URL, params=RESUMEN_PARAMS)
    resumen_data = resumen_response.json()

    # Obtener la URL de la imagen principal de la página
    IMAGEN_PARAMS = {
        "action": "query",
        "format": "json",
        "titles": titulo,
        "prop": "pageimages",
        "pithumbsize": 500  # Tamaño de la miniatura de la imagen
    }

    imagen_response = S.get(url=URL, params=IMAGEN_PARAMS)
    imagen_data = imagen_response.json()

    resumen = list(resumen_data['query']['pages'].values())[0].get('extract', '')
    imagen_url = list(imagen_data['query']['pages'].values())[0].get('thumbnail', {}).get('source', '')

    return resumen, imagen_url

In [10]:
for index, row in df_aerolineas_unicas.iterrows():
    # Verifica si la fila no tiene datos en 'Resumen' o 'Imagen_URL'
    if pd.isnull(row['Resumen']) or row['Resumen'] == '' or pd.isnull(row['Imagen_URL']) or row['Imagen_URL'] == '':
        resumen, imagen_url = obtener_info_wikipedia(row['aerolinea'], idioma='en')  # Cambia a inglés
        df_aerolineas_unicas.at[index, 'Resumen'] = resumen if resumen != '' else df_aerolineas_unicas.at[index, 'Resumen']
        df_aerolineas_unicas.at[index, 'Imagen_URL'] = imagen_url if imagen_url != '' else df_aerolineas_unicas.at[index, 'Imagen_URL']

# Verificar el DataFrame actualizado
print(df_aerolineas_unicas.head())


          aerolinea                                            Resumen  \
0         Envoy Air  Envoy Air[2]​ (anteriormente American Eagle Ai...   
1  SkyWest Airlines  SkyWest Airlines, Inc. es una de las dos aerol...   
2      PSA Airlines  PSA Airlines is an American regional airline h...   
3      Endeavor Air  Endeavor Air is an American regional airline h...   
4     Mesa Airlines  Mesa Airlines, Inc. (NASDAQ: MESA) es una aero...   

                                          Imagen_URL  
0  https://upload.wikimedia.org/wikipedia/commons...  
1  https://upload.wikimedia.org/wikipedia/commons...  
2                                                     
3  https://upload.wikimedia.org/wikipedia/commons...  
4  https://upload.wikimedia.org/wikipedia/commons...  


In [18]:

# URL de la imagen que quieres añadir
url_imagen_psa = "https://en.wikipedia.org/wiki/PSA_Airlines#/media/File:Dornier_328-110,_US_Airways_Express_(PSA_Airlines)_AN0175474.jpg"

# Actualizar la columna 'Imagen_URL' para 'PSA Airlines'
df_aerolineas_unicas.loc[df_aerolineas_unicas['aerolinea'] == 'PSA Airlines', 'Imagen_URL'] = url_imagen_psa

# Verificar el cambio
print(df_aerolineas_unicas[df_aerolineas_unicas['aerolinea'] == 'PSA Airlines'])


      aerolinea                                            Resumen  \
2  PSA Airlines  PSA Airlines is an American regional airline h...   

                                          Imagen_URL  
2  https://en.wikipedia.org/wiki/PSA_Airlines#/me...  


In [21]:
df_aerolineas_unicas.to_pickle('C:/Users/34670/Desktop/python/Hack a boss/proyecto_3/streamlit/proyecto_aviones/data/aerolineas_unicos.pkl')

In [14]:
df_aerolineas_unicas.to_excel('C:/Users/34670/Desktop/python/Hack a boss/proyecto_3/streamlit/proyecto_aviones/data/aerolineas_unicos.xlsx')

In [20]:
df_aerolineas_unicas = pd.read_excel(r'C:/Users/34670/Desktop/python/Hack a boss/proyecto_3/streamlit/proyecto_aviones/data/aerolineas_unicos_2.xlsx')

In [22]:
df_aerolineas_unicas.columns

Index(['aerolinea', 'Resumen', 'IATA', 'ICAO', 'country', 'founded',
       'started_operations', 'Group', 'Base', 'fleet_size',
       'average_fleet_Age', 'official_site'],
      dtype='object')